In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle
from collections import defaultdict
import gc, sys

In [ ]:
with open('../input/m5-pointforecast-itstorder-hist-val-2/ordered_train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)
with open('../input/m5-pointforecast-itstorder-hist-val-2/ordered_val_df.pkl', 'rb') as f:
    val_df = pickle.load(f)
with open('../input/m5-pointforecast-itstorder-hist-val-2/val_weights.pkl', 'rb') as f:
    val_weights = pickle.load(f)

In [ ]:
train_df.shape, val_df.shape

In [ ]:
class Category:
    def __init__(self, unique_items, item_to_id={}):
        self.items = unique_items
        self.item_to_id = item_to_id
        for _id, item_name in enumerate(self.items):
            self.item_to_id[item_name] = _id
        self.type = np.int16
        if len(self.items) > 30000:
            self.type = np.int32
    
    def encode_series(self, items):
        return np.array([self.item_to_id[_item] for _item in items]).astype(self.type)

In [ ]:
with open('../input/m5-point-forecast-createinput-1100/cat_feats.pkl', 'rb') as f:
    cat_feats = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1100/cat_feats_name2idx.pkl', 'rb') as f:
    cat_feats_name2idx = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1100/cat_feats_objs.pkl', 'rb') as f:
    cat_feats_objs = pickle.load(f)

In [ ]:
with open('../input/m5-pointforecast-agggroups-idxs-k-val/agg_groups_idxs_K', 'rb') as f:
    agg_groups = pickle.load(f)

In [ ]:
mask = (val_weights.weight != 0).values

In [ ]:
w = val_weights.weight.values.astype(np.float64)

In [ ]:
w = w[mask]

In [ ]:
K = val_weights.K.values.astype(np.float64)

In [ ]:
K = K[mask]

In [ ]:
K.min(), K.max()

In [ ]:
#sales_mean = val_weights.sales_mean.values[mask]

In [ ]:
#sales_std = val_weights.sales_std.values[mask]

In [ ]:
num_is = (val_weights.weight != 0).sum(); num_is

In [ ]:
import lightgbm as lgb

In [ ]:
gc.collect()

In [ ]:
y_train = train_df.y.values.astype(np.float64)
y_val = val_df.y.values.astype(np.float64)

In [ ]:
y_train, y_val

In [ ]:
X_train_df = train_df.drop('y', axis=1)
X_val_df = val_df.drop('y', axis=1)

In [ ]:
train_dset = lgb.Dataset(X_train_df, y_train)

In [ ]:
val_dset = lgb.Dataset(X_val_df, y_val, reference=train_dset)

In [ ]:
agg_groups[0]

In [ ]:
def custom_obj(preds, train_ds):
    #calculate L for each (item,store)
    y = train_ds.get_label()
    yt = y.reshape(num_is, -1)
    predst = preds.reshape(num_is, -1)
    
    L = np.sum(((yt - predst)**2), axis=1)
    L = L[:,None]
    h = float(yt.shape[1])
    _w = 1.0 * (0.0 + 1e2*w)
    _w = _w/12.0
    grads = -1.0 * _w[:,None] * K[:,None] * (yt - predst) * (h**-0.5) * (L**-0.5)
    hess = (-1.0) * K[:,None] * _w[:,None] * (h**-0.5) * ((-1.0)*(L**-0.5) + ((yt - predst)**2)*(L**-1.5))
    # iterate over agg_groups each one is a single time series
    for idxs_g, K_g in agg_groups:
        yt_g = yt[idxs_g,:]
        predst_g = predst[idxs_g,:]
        _w_g = _w[idxs_g].sum()
        L_g = np.sum((yt_g - predst_g)**2)
        grads_g = -1.0 * _w_g * K_g * (yt_g - predst_g) * (h**-0.5) * (L_g**-0.5) 
        grads[idxs_g,:] = grads[idxs_g,:] + grads_g
        hess_g = (-1.0) * K_g * _w_g * (h**-0.5) * ((-1.0)*(L_g**-0.5) + ((yt_g - predst_g)**2)*(L_g**-1.5))
        hess[idxs_g,:] = hess[idxs_g,:] + hess_g    
    grads = grads.reshape(-1)
    hess = hess.reshape(-1)
    return (grads, hess)

In [ ]:
def custom_eval(preds, train_ds):
    y = train_ds.get_label()
    yt = y.reshape(num_is, -1)
    predst = preds.reshape(num_is, -1)
    #custom_eval
    h = float(yt.shape[1])
    eval_result = np.sum(w * K * (h**-0.5) * (np.sum(((yt-predst)**2), axis=1)**0.5)) 
    for idxs_g, K_g in agg_groups:
        yt_g = yt[idxs_g,:].sum(axis=0)
        predst_g = predst[idxs_g,:].sum(axis=0)
        _w_g = w[idxs_g].sum()
        eval_result += (_w_g * K_g * (np.sum((yt_g-predst_g)**2)**0.5)/ (h**0.5))
    eval_result = eval_result * (1.0/12.0)
    return ('wrmsse', eval_result, False)

In [ ]:
col_names = list(X_train_df.columns.values)

In [ ]:
col_names

In [ ]:
gc.collect()

In [ ]:
# weight_scale 1e2
params = {
    'boosting_type': 'gbdt',
    'num_leaves': 2**10-1,
    'learning_rate': 0.03,
    'metrics':'l2',
    'feature_fraction': 0.7,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'verbose': 2,
    'min_data_in_leaf': 2**11-1,
    'boost_from_average': False
}

gbm = lgb.train(params, train_dset, valid_sets=[val_dset, train_dset], feature_name=col_names, categorical_feature=[0,1,2,3,4],
                fobj=custom_obj, feval=custom_eval, num_boost_round=1200)

In [ ]:
gc.collect()

In [ ]:
sorted(list(zip(col_names, gbm.feature_importance())), key=lambda item: item[1], reverse=True)

In [ ]:
gbm.save_model('gbm_customObj.txt')

In [ ]:
! ls -alrh